# Importação de dependencias

In [26]:
import pandas as pd
import seaborn as sns
import plotly.express as px
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
import numpy as np

# Obter dados através da api da Info Dengue
Documentação: [Tutorial da API](https://info.dengue.mat.br/services/tutorial/Python)


In [27]:
url = "https://info.dengue.mat.br/api/alertcity"
geocode = 2304400
disease = "dengue"
format = "csv"
ew_start = 1
ew_end = 53
ey_start = 2015
ey_end = 2024

params =(
    "&disease="
    + f"{disease}"
    + "&geocode="
    + f"{geocode}"
    + "&disease="
    + f"{disease}"
    + "&format="
    + f"{format}"
    + "&ew_start="
    + f"{ew_start}"
    + "&ew_end="
    + f"{ew_end}"
    + "&ey_start="
    + f"{ey_start}"
    + "&ey_end="
    + f"{ey_end}"
)

url_resp = "?".join([url, params])

In [28]:
dados = pd.read_csv(url_resp, index_col='SE')

# Ler os dados metereologicos pelo site INMET (Instituto nacional de metereologia)
[Link](https://portal.inmet.gov.br/dadoshistoricos)

In [29]:
df = pd.read_csv('./data/FORTALEZA_2015_2024.CSV', encoding='latin1' ,sep=";", decimal=',')

## Limpeza e tratamento de dados

### Substituindo -9999 por np.nan em todo o DataFrame



In [30]:
df = df.replace(-9999, np.nan)

### Formatar Data e criamos uma ramificação só pegando os valores Mensal


In [31]:
df['Data'] = pd.to_datetime(df['Data'],format='%d/%m/%Y')
df.set_index('Data', inplace=True)

#### Condições

In [32]:
condicoes={'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)':"sum",
'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)':"mean",
'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)':"mean",
'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)':"mean",
'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)':"mean",
'TEMPERATURA DO PONTO DE ORVALHO (°C)':"mean",
'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)':"mean",
'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)':"mean",
'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)':"mean",
'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)':"mean",
'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)':"mean",
'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)':"mean",
}


#### Colunas para remover

In [33]:
columns_to_drop_mensal_sem_vento = ['Hora UTC', 'RADIACAO GLOBAL (Kj/m²)',
                   'UMIDADE RELATIVA DO AR, HORARIA (%)',
                   'VENTO, DIREÇÃO HORARIA (gr) (° (gr))',
                   'VENTO, RAJADA MAXIMA (m/s)', 'VENTO, VELOCIDADE HORARIA (m/s)']
df1 = df.copy()

#### Agrupar o dataframe com suas condições

In [34]:
df1 = df1.resample('W').agg(condicoes)

#### Arredondamento

In [35]:
for coluna in df1.columns:
  df1[coluna]=df1[coluna].round(2)

In [36]:
df1['data_iniSE']=df1.index

### Transformar as datas em string

In [37]:
df1['data_iniSE']=df1['data_iniSE'].astype(str)

# Obter informações sobre o DataFrame

In [38]:
df1.head()

,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),data_iniSE
Data,,,,,,,,,,,,,
2015-01-04,54.6,1010.03,1010.28,1009.78,26.45,21.31,26.90,26.07,21.67,20.97,76.16,72.23,2015-01-04
2015-01-11,12.2,1010.34,1010.57,1010.12,27.17,21.27,27.61,26.75,21.72,20.83,73.32,68.60,2015-01-11
2015-01-18,4.8,1009.91,1010.16,1009.67,27.84,21.10,28.26,27.45,21.58,20.67,69.99,65.29,2015-01-18
2015-01-25,3.4,1009.30,1009.52,1009.06,27.79,21.08,28.24,27.38,21.52,20.61,69.74,65.13,2015-01-25
2015-02-01,0.6,1009.20,1009.43,1008.97,27.76,20.92,28.30,27.32,21.38,20.46,69.71,64.88,2015-02-01


In [39]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 523 entries, 2015-01-04 to 2025-01-05
Freq: W-SUN
Data columns (total 13 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                       523 non-null    float64
 1   PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)  493 non-null    float64
 2   PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)        493 non-null    float64
 3   PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)       493 non-null    float64
 4   TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)           493 non-null    float64
 5   TEMPERATURA DO PONTO DE ORVALHO (°C)                   493 non-null    float64
 6   TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)             493 non-null    float64
 7   TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)             493 non-null    float64
 8   TEMPERATURA ORVALHO

In [40]:
df1.columns

Index(['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)',
       'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)',
       'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)',
       'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)',
       'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)',
       'TEMPERATURA DO PONTO DE ORVALHO (°C)',
       'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)',
       'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)',
       'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)', 'data_iniSE'],
      dtype='object')

# Realizar o cruzamento dos dados e limpeza dos dados

In [41]:
cruzamento_dados=df1.merge(dados[['data_iniSE','nivel', 'casos','umidmax', 'tempmax',]],on='data_iniSE',how='left')

In [42]:
cruzamento_dados.dropna(inplace=True)

In [43]:
cruzamento_dados['data_iniSE'] = pd.to_datetime(cruzamento_dados['data_iniSE'],format='%Y-%m-%d')

In [44]:
cruzamento_dados['dia'] = cruzamento_dados['data_iniSE'].dt.day
cruzamento_dados['mes'] = cruzamento_dados['data_iniSE'].dt.month

# Obter informações do cruzamento

In [45]:
cruzamento_dados['nivel'].value_counts()

1.0    274
2.0     85
3.0     76
4.0     42
Name: nivel, dtype: int64

In [46]:
cruzamento_dados.columns

Index(['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)',
       'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)',
       'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)',
       'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)',
       'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)',
       'TEMPERATURA DO PONTO DE ORVALHO (°C)',
       'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)',
       'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)',
       'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)', 'data_iniSE', 'nivel',
       'casos', 'umidmax', 'tempmax', 'dia', 'mes'],
      dtype='object')

## Iniciar os testes

In [47]:
X = cruzamento_dados[[
   'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)',
    'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)',
       'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)',
   'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)',
      'dia',
    'mes'
]]

## Criar lags dsa variáveis


In [48]:
met_vars = [
    'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)',
    'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)',
    'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)',
    'mes',
    'dia'
]
for var in met_vars:
    for lag in [1,2,3,4]:
        cruzamento_dados[f'{var}_lag{lag}'] = cruzamento_dados[var].shift(lag)

### Limpar dados vazios

In [49]:
cruzamento_dados.dropna(inplace=True)

In [50]:
def is_met_var(var_name):
    for met in met_vars:
        if var_name == met or var_name.startswith(met + "_lag"):
            return True
    return False
features = [c for c in cruzamento_dados.columns if is_met_var(c)]
X = cruzamento_dados[features]
y = cruzamento_dados['nivel']

In [51]:
n = len(cruzamento_dados)
n_train = int(0.7 * n)
X_train = X.iloc[:n_train]
X_test  = X.iloc[n_train:]
y_train = y.iloc[:n_train]
y_test  = y.iloc[n_train:]

# Árvore de decição de classificação

In [52]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)

DecisionTreeClassifier()

In [53]:
resultado = decision_tree.predict(X_test)

In [54]:
metrics.recall_score(y_test, resultado, average="micro")

0.5774647887323944

In [55]:
print(classification_report(y_test, resultado))

              precision    recall  f1-score   support

         1.0       0.90      0.81      0.85        79
         2.0       0.27      0.23      0.25        26
         3.0       0.34      0.44      0.39        25
         4.0       0.06      0.08      0.07        12

    accuracy                           0.58       142
   macro avg       0.39      0.39      0.39       142
weighted avg       0.62      0.58      0.59       142



In [56]:
resultado

array([1., 1., 3., 3., 3., 3., 1., 1., 4., 3., 3., 2., 3., 3., 2., 2., 2.,
       2., 2., 2., 4., 2., 2., 1., 3., 2., 3., 2., 3., 3., 4., 3., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 3., 1., 3., 3., 3., 3., 2.,
       2., 2., 3., 4., 2., 3., 4., 3., 2., 4., 4., 4., 1., 4., 4., 1., 2.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 2., 3., 3., 3., 3.,
       3., 3., 3., 2., 2., 2., 4., 3., 3., 4., 4., 4., 4., 4., 4., 1., 1.,
       1., 1., 1., 3., 1., 3.])

# Random Forest

In [57]:
valores=pd.DataFrame(columns=['indice','porcentagem'])
random_forest = RandomForestClassifier(n_estimators=18)
random_forest.fit(X_train, y_train)
resultado = random_forest.predict(X_test)
resultado_porcentagem=metrics.recall_score(y_test, resultado, average="micro")
print(classification_report(y_test, resultado))
valores.loc[len(valores)] = [18, resultado_porcentagem]
print(18,metrics.recall_score(y_test, resultado, average="micro"))

              precision    recall  f1-score   support

         1.0       0.87      0.91      0.89        79
         2.0       0.23      0.19      0.21        26
         3.0       0.56      0.56      0.56        25
         4.0       0.33      0.33      0.33        12

    accuracy                           0.67       142
   macro avg       0.50      0.50      0.50       142
weighted avg       0.65      0.67      0.66       142

18 0.6690140845070423
